# Regression Decision Tree: Low-Level Operations Implementation

Import essential libraries 

In [ ]:
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel
from datetime import datetime
import numpy as np
import geopandas as gpd
from geopy.distance import geodesic
from shapely.geometry import Point
import math
import os, shutil
from hdfs import InsecureClient

Create a Spark Context variable

In [2]:
sc = SparkContext("local[*]", "Low_Level_Regression")
print(sc.appName)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 08:38:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Low_Level_Regression


View the maximum number of workers.

In [3]:
sc.defaultParallelism

8

In [4]:
# Read the GeoJSON file of NYC
nyc = gpd.read_file("new-york-city-boroughs.geojson")

# Convert NYC regions into a list of Polygons
nyc_polygons = nyc['geometry'].tolist()
broadcast_nyc = sc.broadcast(nyc_polygons)

### I. Read the dataset

In [ ]:
# Load the CSV file as a text file and filter out the header
lines = sc.textFile("hdfs:///hcmus/22120210/Lab3/data/train.csv")
lines.take(5)

['id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration',
 'id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982154846191406,40.767936706542969,-73.964630126953125,40.765602111816406,N,455',
 'id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415344238281,40.738563537597656,-73.999481201171875,40.731151580810547,N,663',
 'id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979026794433594,40.763938903808594,-74.005332946777344,40.710086822509766,N,2124',
 'id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040283203125,40.719970703125,-74.01226806640625,40.706718444824219,N,429']

### II. Parse the dataset

Parse the lines from the dataset

In [6]:
header = lines.first().split(',')
data = lines.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])

### III. Pre-processing the data

##### 1. Select columns from the parsed data

The `store_and_fwd_flag` feature indicates whether the trip record was stored in the vehicle's memory before being sent to the vendor, due to the vehicle not having a connection to the server. 'Y' means store and forward, while 'N' means the trip was not a store and forward trip. Since this feature does not provide significant value in predicting trip duration, we have chosen to remove it from the dataset.

In [7]:
# selecte features to convert to float
float_cols = [
'vendor_id',
 'passenger_count',
 'pickup_longitude',
 'pickup_latitude',
 'dropoff_longitude',
 'dropoff_latitude',
 'trip_duration'
]

# Get the selected features' indices in the shown above header
float_indices = [header.index(col) for col in float_cols]
broadcast_indices = sc.broadcast(float_indices)

Parse the data and add more features to detect outliers.

In [8]:
def parse_data(line):
    float_indices = broadcast_indices.value
    try:
        parts = line.strip().split(',')
        
        # TIME FEATURES
        dt_start_str = parts[2]  # pick up time
        dt_end_str = parts[3]  # drop off time
        dt_start = datetime.strptime(dt_start_str, "%Y-%m-%d %H:%M:%S")
        dt_end = datetime.strptime(dt_end_str, "%Y-%m-%d %H:%M:%S")
        trip_duration = (dt_end - dt_start).total_seconds() # Add trip duration feature for later consistency checking
        
        # Extract feature: hour, weekday, day of year
        time_features = [dt_start.hour, dt_start.weekday(), dt_start.timetuple().tm_yday]

        # Parse numerical values
        parsed = [float(parts[i]) for i in float_indices]
        
        return tuple(parsed + time_features + [trip_duration])
    except Exception as e:
        print(f"Error processing line: {line} -> {e}")
        return None  # Print log's information and return None when encounter error in processing

Remove inconsisent rows and duplicated rows in dataset 

In [9]:
parsedData = data.map(parse_data).filter(lambda x: x is not None)
                        
# Remove rows that have trip duration not equal to 
# result calculated from pick up date time and drop off date time
rdd_data = parsedData.filter(lambda x: int(x[6]) == int(x[10]))

rdd_data = rdd_data.distinct()

In [10]:
rdd_data.take(1)[0]

(2.0,
 1.0,
 -74.01004028320312,
 40.719970703125,
 -74.01226806640625,
 40.70671844482422,
 429.0,
 19,
 2,
 97,
 429.0)

Because there is no error log on the cell above (which performs converting datatype from string to float), we assume that the dataset does not have `NaN` value. 

##### 2. Remove rows with invalid pick up/drop off location

In [11]:
def is_in_nyc(lon, lat):
    pt = Point(lon, lat)
    for polygon in broadcast_nyc.value:
        if polygon.contains(pt):
            return True
    return False

def check_row(row):
    """Remove rows that have neither pick up location nor drop off location in New York City"""
    pickup_in = is_in_nyc(row[2], row[3])
    dropoff_in = is_in_nyc(row[4], row[5])
    return pickup_in or dropoff_in

rdd_data = rdd_data.filter(check_row).persist(StorageLevel.MEMORY_AND_DISK)

Split the dataset to train/test set

In [12]:
train_rdd, test_rdd = rdd_data.randomSplit([0.7,0.3], seed=42)

Now we will add some more features to train set to detect outliers.

In [13]:
def add_features_data(line):
    try:
        # SPATIAL FEATURE
        start_location = (line[3], line[2])
        end_location = (line[5], line[4])
        direct_distance = geodesic(start_location, end_location).meters # Add direct distance feature for later logic checking
        
        # VELOCITY FEATURE 
        velocity = direct_distance / line[-1] * 3.6 # Add velocity feature for later logic checking (km/h)
        return line[:6] + line[7:] + (direct_distance, velocity)
    except Exception as e:
        print(f"Error processing line: {line} -> {e}")
        return None  # Print log's information and return None when encounter error in processing
    
train_rdd = train_rdd.map(add_features_data)

In [14]:
train_rdd = train_rdd.persist(StorageLevel.MEMORY_AND_DISK)

All features of the training set are listed below in order:
- vendor id
- passenger count
- pickup longitude
- pickup latitude
- dropoff longitude
- dropoff latitude
- hour
- weekday
- yearday
- trip duration
- direct distance
- velocity

##### 3. Remove outliers

As shown in the referenced EDA, the columns we need to preprocess are all non-normally distributed. Therefore, we need to use appropriate methods to detect and remove as many outliers as possible.

Additionally, we need to define a function for statistical reporting to ensure that these methods are on the right track.

In [15]:
# Hàm thu thập giá trị cột
def collect_column_values(row):
    num_columns = num_columns_broadcast.value
    offset = offset_broadcast.value
    return [(col_idx, [row[col_idx]]) for col_idx in range(num_columns - offset, num_columns)]

def statistical_report(values):
    values = sorted([float(v) for v in values])
    n = len(values)
    
    min_val = values[0]
    max_val = values[-1]
    mean_val = sum(values) / n

    # Tính median
    if n % 2 == 1:
        median_val = values[n // 2]
    else:
        median_val = (values[n // 2 - 1] + values[n // 2]) / 2

    return (min_val, max_val, mean_val, median_val)

We collect values at each selected column.

In [16]:
# Because we want to collect the last 3 columns, we create 
# a broadcast variable to pass to collect_column_values function
offset_broadcast = sc.broadcast(3)
# Number of columns
num_columns_broadcast = sc.broadcast(len(train_rdd.take(1)[0]))

# Collect values grouped by column
col_values_rdd = train_rdd.flatMap(collect_column_values)
col_values_grouped = col_values_rdd.reduceByKey(lambda x, y: x + y).persist(StorageLevel.MEMORY_AND_DISK)

25/05/10 10:19:10 WARN BlockManager: Task 15 already completed, not releasing lock for rdd_11_0


First, we come with IQR approach:

In [17]:
def calculate_bounds(values):
    # Convert to float and sort
    values = sorted([float(v) for v in values])
    n = len(values)
    
    def percentile(p):
        k = (n - 1) * (p / 100)
        f = int(k)
        c = f + 1
        if c >= n:
            return values[f]
        d0 = values[f] * (c - k)
        d1 = values[c] * (k - f)
        return d0 + d1

    Q1 = percentile(25)
    Q3 = percentile(75)
    IQR = Q3 - Q1
    return (Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)

# Calculate the bounds concurrently
bounds_rdd = col_values_grouped.mapValues(calculate_bounds)
bounds = bounds_rdd.collectAsMap()

After calculating lower bound and upper bound of each column, let's take a look at them (to check whether the bounds are reasonable or not)

In [18]:
# Print all the bounds
for col_idx, (lower, upper) in bounds.items():
    print(f"Column {col_idx}: Lower bound = {lower}, Upper bound = {upper}")

Column 9: Lower bound = -618.5, Upper bound = 2089.5
Column 10: Lower bound = -2725.0618484028378, Upper bound = 7828.676667884347
Column 11: Lower bound = -3.9613838811481177, Upper bound = 30.93929904258277


Columns 9, 10, and 11 sequentially represent `trip duration`, `direct distance`, and `velocity`. To be honest, a `trip duration` of more than 2000 seconds (~33 minutes) cannot be considered an outlier. The lower bound is a negative float, so there is nothing to analyze in that regard. I'm not entirely sure how the infrastructure in New York City is distributed, but a `direct distance` greater than 7.8 km can be considered a tolerable upper bound. The upper bound for `velocity` might be reasonable because, given the traffic conditions in NYC, an average speed of 31 km/h (which would likely be higher in reality) is not achievable. Therefore, we will try applying the IQR method for both `velocity` and `direct distance`.

In [19]:
# Filter all the outlies
def IQR_outlier(row):
    num_columns = num_columns_broadcast.value
    offset = offset_broadcast.value
    for col_idx in range(num_columns - offset, num_columns):
        value = row[col_idx]
        lower_bound, upper_bound = bounds[col_idx]
        if value < lower_bound or value > upper_bound:
            return False
    return True

offset_broadcast = sc.broadcast(2)
filtered_iqr_rdd = train_rdd.filter(IQR_outlier)

Now, let's print the statistical report after applying IQR for the dataset

In [20]:
name_col = ["Direct Distance", "Velocity"]
# Because we want to collect the last 2 columns so we create 
# a broadcast variable to pass to collect_column_values function
offset_broadcast = sc.broadcast(2)

# collect values grouped by columns
col_iqr_rdd = filtered_iqr_rdd.flatMap(collect_column_values)
col_iqr_grouped = col_iqr_rdd.reduceByKey(lambda x, y: x + y)

# Calculate statistical report for each column 
iqr_filtered_rdd = col_iqr_grouped.mapValues(statistical_report)
iqr_report = iqr_filtered_rdd.collectAsMap()

# Print the statistical report for IQR filtered column
for i in range(2):
    print(f"\n-----{name_col[i]}-----")
    report = iqr_report[num_columns_broadcast.value - 2 + i]
    print(f"Min: {report[0]}")
    print(f"Max: {report[1]}")
    print(f"Mean: {report[2]}")
    print(f"Median: {report[3]}")


-----Direct Distance-----
Min: 0.0
Max: 7828.372302664765
Mean: 2356.09453026498
Median: 1889.4171528723853

-----Velocity-----
Min: 0.0
Max: 30.939142108169577
Mean: 12.776125067547364
Median: 11.996498056419297


The lower bound is not good enough to handle those outliers. Maybe we should try another approach.

Next, we move to robust Z-score approach

In [21]:
def collect_column_values_abs_med(row):
    num_columns = num_columns_broadcast.value
    meds = meds_broadcast.value
    return [(col_idx, [abs(row[col_idx] - meds[col_idx])]) for col_idx in range(num_columns - 3, num_columns)]

def calculate_med(values):
    values = sorted([float(v) for v in values])
    n = len(values)
    
    def percentile(p):
        k = (n - 1) * (p / 100)
        f = int(k)
        c = f + 1
        if c >= n:
            return values[f]
        d0 = values[f] * (c - k)
        d1 = values[c] * (k - f)
        return d0 + d1
    return percentile(50)

# Calculate median of each column concurrently
meds_rdd = col_values_grouped.mapValues(calculate_med)
meds = meds_rdd.collectAsMap()

meds_broadcast = sc.broadcast(meds)

# Collect values grouped by columns
col_values_abs_med_rdd = train_rdd.flatMap(collect_column_values_abs_med)
col_values_abs_med_grouped = col_values_abs_med_rdd.reduceByKey(lambda x, y: x + y)

# Calculate MAD (Median Absolute Deviation) of each column concurrently
mads_rdd = col_values_abs_med_grouped.mapValues(calculate_med)
mads = mads_rdd.collectAsMap()

In [22]:
col_values_grouped.unpersist()

PythonRDD[17] at RDD at PythonRDD.scala:53

Filter the outliers in selected columns

In [23]:
# Filter the outliers
def robust_column_rdd(row):
    num_columns = num_columns_broadcast.value
    meds = meds_broadcast.value
    mads = mads_broadcast.value
    return [(col_idx, abs(row[col_idx] - meds[col_idx]) / mads[col_idx]) 
            for col_idx in range(num_columns - 3, num_columns)]
    
robust_rdd = train_rdd.flatMap(robust_column_rdd).reduceByKey(lambda x,y: x + y)

# Filter the outliers
def robust_outlier(row):
    num_columns = num_columns_broadcast.value
    meds = meds_broadcast.value
    mads = mads_broadcast.value
    # threshold = threshold_broadcast.value
    for col_idx in range(num_columns - 3, num_columns):
        if abs(row[col_idx] - meds[col_idx]) / mads[col_idx] > 3:
            return False
    return True

mads_broadcast = sc.broadcast(mads)
# threshold_broadcast = sc.broadcast(threshold)
filtered_robust_rdd = train_rdd.filter(robust_outlier)

Now, let's print the statistical report after applying robust Z-score for the dataset

In [24]:
name_col = ["Trip duration","Direct Distance", "Velocity"]
# Because we want to collect the last 3 columns so we create 
# a broadcast variable to pass to collect_column_values function
offset_broadcast = sc.broadcast(3)

# collect value grouped by columned
col_robust_rdd = filtered_robust_rdd.flatMap(collect_column_values)
col_robust_grouped = col_robust_rdd.reduceByKey(lambda x, y: x + y)

# Calculate statistical report for each column 
robust_filtered_rdd = col_robust_grouped.mapValues(statistical_report)
robust_report = robust_filtered_rdd.collectAsMap()

# Print the statistical report for IQR filtered column
for i in range(3):
    print(f"\n-----{name_col[i]}-----")
    report = robust_report[num_columns_broadcast.value - 3 + i]
    print(f"Min: {report[0]}")
    print(f"Max: {report[1]}")
    print(f"Mean: {report[2]}")
    print(f"Median: {report[3]}")


-----Trip duration-----
Min: 1.0
Max: 1595.0
Mean: 624.868964779068
Median: 569.0

-----Direct Distance-----
Min: 0.4236109090686456
Max: 5277.969420283912
Mean: 1981.7221711158975
Median: 1723.4443160613012

-----Velocity-----
Min: 0.2577348925578789
Max: 25.343768423001187
Mean: 12.202033285699084
Median: 11.678126558060292


Based on the statistical report, looks like it even get worse. Because the `velocity` and `direct distance` cannot be equal to zero, we can consider it as outliers.

In [25]:
cleaned_rdd = filtered_iqr_rdd.filter(lambda x: x[-1] > 0 and x[-1] > 0)

In [26]:
# Create an RDD where each record is a tuple: (features as a list, label)
cleaned_data = cleaned_rdd.map(lambda cols: (cols[:-3], cols[-3])).persist(StorageLevel.MEMORY_AND_DISK)

first_element = cleaned_data.take(1)[0]
print(first_element)
print(type(first_element)) 
print(type(first_element[0]))
print(type(first_element[1]))
print(cleaned_data.count())

25/05/10 10:43:40 WARN BlockManager: Task 70 already completed, not releasing lock for rdd_39_0


((2.0, 1.0, -74.01004028320312, 40.719970703125, -74.01226806640625, 40.70671844482422, 19, 2, 97), 429.0)
<class 'tuple'>
<class 'tuple'>
<class 'float'>
889018


In [27]:
# cleaned_data consist of lazy evaluation operation on train_rdd, so we have to make sure that 
# train_rdd is not unpersisted before cleaned_data call a non lazy evaluation operation and being cached
train_rdd.unpersist() 

PythonRDD[11] at RDD at PythonRDD.scala:53

### III. Train the model.

In [28]:
cleaned_data.count()

889018

In [ ]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

In [ ]:
# Compute adaptive quantiles based on dataset size
def get_adaptive_quantiles(rdd, feature_index, mode='sqrt'):
    count = rdd.count()

    if mode == 'sqrt':
        num_quantiles = int(math.sqrt(count))
    elif mode == 'log':
        num_quantiles = int(math.log2(count))
    else:
        num_quantiles = 10  # fallback

    values = rdd.map(lambda x: x[0][feature_index]) \
                .distinct() \
                .sortBy(lambda x: x) \
                .collect()

    if len(values) <= num_quantiles:
        return values

    step = len(values) // (num_quantiles + 1)
    quantiles = [values[i * step] for i in range(1, num_quantiles + 1)]
    return quantiles

In [31]:
# Function to find best split on RDD
def find_best_split_rdd(rdd, num_features):
    def evaluate_split(feature_index):
        quantiles = get_adaptive_quantiles(rdd, feature_index)

        best_mse = float('inf')
        best_threshold = None

        for threshold in quantiles:
            # Return values of seq_op: sumL, sq_sumL, countL, sumR, sq_sumR, countR
            def seq_op(acc, row):
                value = row[0][feature_index]
                label = row[1]
                if value <= threshold:
                    return (
                        acc[0] + label,           # sumL
                        acc[1] + label ** 2,      # sq_sumL
                        acc[2] + 1,               # countL
                        acc[3], acc[4], acc[5]    # right remains the same
                    )
                else:
                    return (
                        acc[0], acc[1], acc[2],   # left remains the same
                        acc[3] + label,           # sumR
                        acc[4] + label ** 2,      # sq_sumR
                        acc[5] + 1                # countR
                    )

            def comb_op(acc1, acc2):
                return tuple(a + b for a, b in zip(acc1, acc2))

            # Initialized values
            init = (0.0, 0.0, 0, 0.0, 0.0, 0)
            sumL, sq_sumL, countL, sumR, sq_sumR, countR = rdd.aggregate(init, seq_op, comb_op)

            if countL == 0 or countR == 0:
                continue

            mse_left = (sq_sumL / countL) - (sumL / countL) ** 2
            mse_right = (sq_sumR / countR) - (sumR / countR) ** 2
            total_mse = (countL * mse_left + countR * mse_right) / (countL + countR)

            if total_mse < best_mse:
                best_mse = total_mse
                best_threshold = threshold

        return (feature_index, best_threshold, best_mse)


    results = [evaluate_split(i) for i in range(num_features)]

    # Choose the best split
    best_result = min(results, key=lambda x: x[2])
    return best_result

In [32]:
# Build tree
def build_tree_rdd(rdd, num_features, depth=0, max_depth=3, min_samples=10):
    rdd.persist(StorageLevel.MEMORY_AND_DISK)
    count = rdd.count()
    
    # Stop condition
    if count < min_samples or depth >= max_depth:
        mean = rdd.map(lambda x: x[1]).reduce(lambda x, y: x + y) / count
        return Node(value=mean)
    
    # Find optimal split
    feature_index, threshold, mse = find_best_split_rdd(rdd, num_features)
    
    if feature_index is None or threshold is None:
        mean = rdd.map(lambda x: x[1]).reduce(lambda x, y: x + y) / count
        return Node(value=mean)
    
    # Split the RDD
    left_rdd = rdd.filter(lambda x: x[0][feature_index] <= threshold)
    right_rdd = rdd.filter(lambda x: x[0][feature_index] > threshold)
    
    # Build the branch recuresively
    left_node = build_tree_rdd(left_rdd, num_features, depth + 1, max_depth, min_samples)
    right_node = build_tree_rdd(right_rdd, num_features, depth + 1, max_depth, min_samples)
    
    rdd.unpersist()
    
    return Node(feature_index=feature_index, threshold=threshold, 
                left=left_node, right=right_node)


# Build the tree
num_features = len(cleaned_data.take(1)[0][0])
min_samples = cleaned_data.count() // 100
tree = build_tree_rdd(cleaned_data, num_features, 0, 3, min_samples)

25/05/10 10:43:41 WARN BlockManager: Task 83 already completed, not releasing lock for rdd_39_0


In [33]:
# Create an RDD where each record is a tuple: (features as a list, label)
test_data = test_rdd.map(lambda cols: (cols[:6] + cols[7:-1], cols[-1]))

first_element = test_data.take(1)[0]
print(first_element)
print(type(first_element)) 
print(type(first_element[0]))
print(type(first_element[1]))
test_size =test_data.count()
print(test_size)

25/05/10 11:22:14 WARN BlockManager: Task 100032 already completed, not releasing lock for rdd_10_0


((1.0, 1.0, -74.00899505615234, 40.712589263916016, -73.9986343383789, 40.722808837890625, 12, 5, 9), 707.0)
<class 'tuple'>
<class 'tuple'>
<class 'float'>
437341


In [34]:
rdd_data.unpersist()

PythonRDD[10] at RDD at PythonRDD.scala:53

In [35]:
# Hàm dự đoán
def predict(tree, sample):
    if tree.value is not None:
        return tree.value
    if sample[tree.feature_index] <= tree.threshold:
        return predict(tree.left, sample)
    return predict(tree.right, sample)

prediction_rdd = test_data.map(lambda x: (x[1], predict(tree, x[0]))).persist(StorageLevel.MEMORY_AND_DISK)

#### IV. Evaluate the model.

In [ ]:
def compute_rmse(prediction_rdd):
    sum_squared_error, count = prediction_rdd.aggregate(
        (0.0, 0),
        lambda acc, x: (acc[0] + (x[0] - x[1]) ** 2, acc[1] + 1),
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])
    )
    mse = sum_squared_error / count
    rmse = math.sqrt(mse)
    return rmse

rmse = compute_rmse(prediction_rdd)
print(f"RMSE: {rmse}")

RMSE: 4608.925435997215


In [ ]:
def r2_seq_op(acc, row):
    y = row[0]
    y_hat = row[1]
    return (
        acc[0] + y,            # sum y for mean computing
        acc[1] + 1,            # count
        acc[2] + (y - y_hat) ** 2,     # ss_res
        acc[3] + y ** 2                 # ss_tot for later variance computing
    )

def r2_comb_op(a, b):
    return tuple(x + y for x, y in zip(a, b))

sum_y, count, ss_res, sum_y_sq = prediction_rdd.aggregate((0.0, 0.0, 0.0, 0.0), r2_seq_op, r2_comb_op)
mean_y = sum_y / count
ss_tot = sum_y_sq - (sum_y ** 2) / count

r2 = 1 - ss_res / ss_tot
print(f"R²: {r2}")

R²: 0.0005965947874605471


In [38]:
# cleaned_data.unpersist
# prediction_rdd.unpersist()

##### V. Predict the data from test.csv file

Let's take a look at the test.csv file.

In [ ]:
# Load the CSV file as a text file and filter out the header
lines = sc.textFile("hdfs:///hcmus/22120210/Lab3/data/train.csv")
lines.take(5)

['id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag',
 'id3004672,1,2016-06-30 23:59:58,1,-73.988128662109375,40.732028961181641,-73.99017333984375,40.756679534912109,N',
 'id3505355,1,2016-06-30 23:59:53,1,-73.964202880859375,40.67999267578125,-73.959808349609375,40.655403137207031,N',
 'id1217141,1,2016-06-30 23:59:47,1,-73.9974365234375,40.737583160400391,-73.986160278320312,40.729522705078125,N',
 'id2150126,2,2016-06-30 23:59:41,1,-73.956069946289063,40.771900177001953,-73.986427307128906,40.73046875,N']

Parse the test.csv file to RDD

In [51]:

def parse_predict_data(line):
    float_indices = broadcast_indices.value
    try:
        parts = line.strip().split(',')
        
        # TIME FEATURES
        dt_start_str = parts[2]  # pick up time
        dt_start = datetime.strptime(dt_start_str, "%Y-%m-%d %H:%M:%S")
        
        # Extract feature: hour, weekday, day of year
        time_features = [dt_start.hour, dt_start.weekday(), dt_start.timetuple().tm_yday]

        # Parse numerical values
        parsed = [float(parts[i]) for i in float_indices]
        
        return parts[0], tuple(parsed + time_features)
    except Exception as e:
        print(f"Error processing line: {line} -> {e}")
        return None  # Print log's information and return None when encounter error in processing
    
predict_sample_header = lines.first().split(',')
predict_sample_rdd = lines.zipWithIndex().filter(lambda x: x[1] > 0).map(lambda x: x[0])

Reformat the parse data for predicting

In [52]:
# Get the selected features' indices in the shown above header
float_indices = [predict_sample_header.index(col) for col in float_cols[:-1]]
broadcast_indices = sc.broadcast(float_indices)

predict_data = predict_sample_rdd.map(parse_predict_data).filter(lambda x: x is not None)

Predicting and processing the output file

In [ ]:
final_predict = predict_data.map(lambda x: (x[0], predict(tree, x[1])))

# 1. Create header
header = [("id", "trip_duration")]

# 2. Merge header with the result
final_with_header = sc.parallelize(header).union(final_predict)

# 3. Convert each row to CSV string
csv_rdd = final_with_header.map(lambda row: ",".join(map(str, row)))

# 4. Save as text file
csv_rdd.coalesce(1).saveAsTextFile("hdfs:///hcmus/22120210/Lab3/output_dir")


In [ ]:
output_dir = "hdfs:///hcmus/22120210/Lab3/output_dir"
final_file = "hdfs:///hcmus/22120210/Lab3/result.csv"

# Use HDFS file system (ensure Hadoop is set up and pyarrow or hdfs library is installed)
hdfs_client = InsecureClient('http://localhost:9870', user='khtn_22120210')

# Check whether the output file has already existed or not, if yes remove the existed file
if hdfs_client.status(final_file, strict=False):  # This checks if the file exists
    hdfs_client.delete(final_file)

# Move part-* and change its name as desired
for f in os.listdir(output_dir):
    if f.startswith("part-"):
        # Read the part- file, write its content into final result
        with hdfs_client.write(final_file, overwrite=True) as writer:
            with open(os.path.join(output_dir, f), 'rb') as reader:
                shutil.copyfileobj(reader, writer)
        break

# Remove temporary folder on HDFS
hdfs_client.delete(output_dir, recursive=True)
